# Поиск объявлений по реновации, размещенных на ЦИАН
Мы хотим найти на ЦИАН квартиры, которые продаются в домах из программы реновации. Зачем? Во-первых, это может быть полезно тем, кто следит за темой реновации - например, хочет купить квартиру в уже утверждённом доме. Во-вторых, в этом сегменте может быть особая динамика по ценам, спросу и предложениям - и если выделить эти объявления, можно сделать отдельный срез рынка.

Проблема в том, что напрямую на ЦИАН такие квартиры часто не помечаются, а иногда и наоборот - пишут, что квартира подпадает под реновацию, но на самом деле не находится в реестре. Поэтому мы идём в обход: берём официальный список домов из реестра реновации, парсим объявления с ЦИАН и пытаемся сопоставить адреса - несмотря на то, что в обоих источниках они записаны по-разному. Это требует немного нормализации, парсинга и частичных сравнений, но в итоге даёт вполне рабочую выборку квартир по реновации.

## 0) Установка зависимостей

Запускаем Playwright и необходимые пакеты. Если вы перезапускаете среду, повторите эту ячейку.

In [1]:
%pip -q install playwright pandas openpyxl requests
!python -m playwright install chromium

,Адрес,ext_status,district,coords
0,"1-й Ботанический проезд, дом 1",None,2556,"[55.8502, 37.634]"
1,"1-й Ботанический проезд, дом 3",None,2556,"[55.851, 37.6343]"
2,"1-й Ботанический проезд, дом 3А",None,2556,"[55.8511, 37.6336]"
3,"1-й Волоколамский проезд, дом 7, корпус 1",destroyed,2483,"[55.8008, 37.4918]"
4,"1-й Волоколамский проезд, дом 7, корпус 2",destroyed,2483,"[55.8007, 37.4911]"


## 1) Импорты и базовые константы
Держим настройки в одном месте: город для парсинга адресов и параметры запроса к официальному реестру реновации.

In [57]:
from __future__ import annotations
import re, asyncio
from typing import Iterable, List, Optional, Tuple

import requests
import pandas as pd

CITY = "москва"

URL = "https://fr.mos.ru/pokupka-nedvizhimosti-dlya-vseh/ajax.php"
PARAMS = {"status[]": "OLD", "pagesize": 100000, "map": "ren"}
HEADERS = {"User-Agent": "Mozilla/5.0"}


## 2) Загрузка реестра реновации
Реестр реновации мы тянем напрямую с сайта fr.mos.ru, где он подгружается динамически через XHR-запрос к ajax.php. Если открыть DevTools в браузере (F12), перейти на вкладку Network → XHR/Fetch и перезагрузить страницу, можно отследить этот запрос и скопировать его параметры. Мы делаем ровно то же самое - обращаемся к тому же эндпоинту, с теми же параметрами и заголовком, чтобы получить сырой JSON с нужными нам объектами.

После запроса достаём список домов (items) из ключей ["objects"]["items"], и на его основе формируем датафрейм с нужными полями: адрес, статус, район и координаты. Это ещё не нормализованные адреса, просто черновой реестр - но уже можно посмотреть, что в нём лежит.

In [58]:
resp = requests.get(URL, params=PARAMS, headers=HEADERS, timeout=30)
resp.raise_for_status()
items = resp.json()["objects"]["items"]

ren_df = pd.DataFrame([
    {
        "Адрес": d.get("name"),
        "ext_status": d.get("ext_status"),
        "district": d.get("district"),
        "coords": d.get("coords"),
    }
    for d in items
])

ren_df.head(5)


,Адрес,ext_status,district,coords
0,"1-й Ботанический проезд, дом 1",None,2556,"[55.8502, 37.634]"
1,"1-й Ботанический проезд, дом 3",None,2556,"[55.851, 37.6343]"
2,"1-й Ботанический проезд, дом 3А",None,2556,"[55.8511, 37.6336]"
3,"1-й Волоколамский проезд, дом 7, корпус 1",destroyed,2483,"[55.8008, 37.4918]"
4,"1-й Волоколамский проезд, дом 7, корпус 2",destroyed,2483,"[55.8007, 37.4911]"


## 3) Парсинг объявлений ЦИАН через Playwright

Используем Playwright (Chromium в headless-режиме): открываем страницы, ждём появления карточек и читаем DOM по CSS-селекторам. Почему так: у ЦИАН нет открытого простого JSON-эндпоинта для всего списка, а HTML стабилен под эти селекторы.

In [59]:
from playwright.async_api import async_playwright

async def parse_cian(pages: int = 2, max_price: int = 7_000_000, save_path: Optional[str] = None) -> pd.DataFrame:
    data = []
    url_template = (
        "https://www.cian.ru/cat.php?engine_version=2&p={page}&deal_type=sale&offer_type=flat&region=1&maxprice={max_price}"
    )

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        for i in range(1, pages + 1):
            url = url_template.format(page=i, max_price=max_price)
            print("Смотрим:", url)
            await page.goto(url, wait_until="domcontentloaded")
            await page.wait_for_selector("article")
            offers = await page.query_selector_all("article")

            async def get_text(el, selector: str) -> str:
                node = await el.query_selector(selector)
                return (await node.inner_text()) if node else ""

            for offer in offers:
                title = await get_text(offer, "span[data-mark='OfferTitle']")
                price = await get_text(offer, "span[data-mark='MainPrice']")
                link_el = await offer.query_selector("a")
                link = await link_el.get_attribute("href") if link_el else ""

                geo_items = await offer.query_selector_all("a[data-name='GeoLabel']")
                address = ", ".join([await g.inner_text() for g in geo_items]) if geo_items else "—"

                data.append({"title": title, "price": price, "Адрес": address, "Ссылка на объявление": link})

        await browser.close()

    df = pd.DataFrame(data)
    if save_path:
        df.to_excel(save_path, index=False)
        print("Сохранено в", save_path)
    return df


### 3.1) Запуск парсера
В данном случае выбраны параметры максимальной цены 10.000.000 для квартир в городе Москве, но при желании код может быть дополнен под более специфичные требования.

In [76]:
offers_df = await parse_cian(pages=250, max_price=10_000_000, save_path="offers.xlsx")
offers_df.head(5)


Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=1&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=2&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=3&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=4&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=5&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=6&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=7&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=8&deal_type=sale&offer_type=flat&region=1&maxprice=10000000
Смотрим: https://www.cian.ru/cat.php?engine_version=2&p=9&deal_type=sale

,title,price,Адрес,Ссылка на объявление
0,Апартаменты с White Box,,"Москва, ЮАО, р-н Чертаново Южное, м. Аннино, В...",https://www.cian.ru/sale/flat/319308727/
1,доступное жилье москва,4 822 000 ₽,"Москва, ЮВАО, р-н Кузьминки, м. Окская, Окская...",https://www.cian.ru/sale/flat/306880520/
2,"Апартаменты-студия, 15,7 м², 1/5 этаж",4 330 000 ₽,"Москва, ЗАО, р-н Солнцево, м. Говорово, улица ...",https://www.cian.ru/sale/flat/317909846/
3,"Апартаменты-студия, 27,3 м², 19/23 этаж",9 229 502 ₽,"Москва, СВАО, р-н Отрадное, м. Владыкино, Сигн...",https://www.cian.ru/sale/flat/309370579/
4,"Студия в ЖК ""My Space в Ховрино""",6 510 000 ₽,"Москва, САО, р-н Головинский, м. Моссельмаш, С...",https://www.cian.ru/sale/flat/311088838/


In [77]:
offers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6993 entries, 0 to 6992
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 6993 non-null   object
 1   price                 6993 non-null   object
 2   Адрес                 6993 non-null   object
 3   Ссылка на объявление  6993 non-null   object
dtypes: object(4)
memory usage: 218.7+ KB


## 4) Поиск совпадений

Чтобы сопоставить адреса из ЦИАН с адресами из реестра реновации, мы пошли не по самому прямолинейному пути. Дело в том, что в этих двух источниках один и тот же дом может быть записан совершенно по-разному: где-то указано "ул. 50 лет Октября, 2К2", где-то "улица 50 лет Октября, дом 2, корпус 1", а где-то всё это сопровождается районом, метро и жилым комплексом. В лоб такие строки не сравнишь.

Поэтому мы упростили: для каждого адреса мы выделили название улицы и основной номер дома (без корпуса, строения и прочего). Если и улица, и номер дома совпадают - считаем, что это один и тот же дом. Такой подход не идеален, но позволяет выловить хотя бы базовые совпадения.

В результате мы нашли 7 совпадений. Это немного, но объяснимо: адреса слишком разномастные, где-то мешает разница в корпусах, где-то - в написании улиц. Например, один адрес на ЦИАН может матчиться сразу с несколькими корпусами из реновации, а некоторые реальные совпадения могли не попасть в выборку из-за слишком разных формулировок.

Что можно попробовать ещё:
- нормализовать названия улиц (заменять "ул." на "улица", "пр-т" на "проспект" и т.п.);
- учитывать корпус, если он явно указан;
- дополнительно применять частичное или нестрогое сравнение (по токенам, как раньше), чтобы найти более неочевидные совпадения.

Но даже такой базовый фильтр уже отсекает кучу нерелевантных адресов и даёт точку опоры для анализа.

In [111]:
def extract_base_house(addr: str) -> str:
    if pd.isna(addr):
        return None
    match = re.search(r'\b(\d{1,4})', addr)
    return match.group(1) if match else None
offers_df[["street", "house_base"]] = offers_df["Адрес"].apply(
    lambda x: pd.Series([*extract_street_and_house(x)] if extract_street_and_house(x) else (None, None))
)
offers_df["house_base"] = offers_df["Адрес"].map(extract_base_house)

ren_df[["street", "house_base"]] = ren_df["Адрес"].apply(
    lambda x: pd.Series([*extract_street_and_house(x)] if extract_street_and_house(x) else (None, None))
)
ren_df["house_base"] = ren_df["Адрес"].map(extract_base_house)
offers_clean = offers_df.dropna(subset=["street", "house_base"])
ren_clean = ren_df.dropna(subset=["street", "house_base"])

matches_base = offers_clean.merge(
    ren_clean[["street", "house_base", "Адрес", "district", "ext_status"]],
    on=["street", "house_base"],
    how="inner",
    suffixes=("_ЦИАН", "_реновация")
)

print(f"Найдено совпадений по базовому дому: {len(matches_base)}")
matches_base[["Адрес_ЦИАН", "Адрес_реновация", "Ссылка на объявление", "district", "ext_status"]].head(10)


Найдено совпадений по базовому дому: 7


,Адрес_ЦИАН,Адрес_реновация,Ссылка на объявление,district,ext_status
0,"Москва, СВАО, р-н Ростокино, м. Ростокино, про...","проспект Мира, дом 188А",https://www.cian.ru/sale/flat/320292048/,2569,None
1,"Москва, СВАО, р-н Ростокино, м. Ростокино, про...","проспект Мира, дом 188А",https://www.cian.ru/sale/flat/320292048/,2569,None
2,"Москва, ЮЗАО, р-н Котловка, м. Крымская, улица...","улица Дмитрия Ульянова, дом 47, корпус 2",https://www.cian.ru/sale/flat/321183005/,2550,destroyed
3,"Москва, СЗАО, р-н Хорошево-Мневники, м. Бульва...","бульвар Генерала Карбышева, дом 6, корпус 1",https://www.cian.ru/sale/flat/319591523/,2507,None
4,"Москва, СЗАО, р-н Хорошево-Мневники, м. Бульва...","бульвар Генерала Карбышева, дом 6, корпус 2",https://www.cian.ru/sale/flat/319591523/,2507,None
5,"Москва, СЗАО, р-н Хорошево-Мневники, м. Бульва...","бульвар Генерала Карбышева, дом 6, корпус 3",https://www.cian.ru/sale/flat/319591523/,2507,None
6,"Москва, СЗАО, р-н Хорошево-Мневники, м. Бульва...","бульвар Генерала Карбышева, дом 6, корпус 4",https://www.cian.ru/sale/flat/319591523/,2507,None


In [114]:
list(matches_base['Адрес_ЦИАН'][0:20])

['Москва, СВАО, р-н Ростокино, м. Ростокино, проспект Мира, 188Бк2',
 'Москва, СВАО, р-н Ростокино, м. Ростокино, проспект Мира, 188Бк2',
 'Москва, ЮЗАО, р-н Котловка, м. Крымская, улица Дмитрия Ульянова, 47',
 'Москва, СЗАО, р-н Хорошево-Мневники, м. Бульвар Генерала Карбышева, Живописная улица, 6К3',
 'Москва, СЗАО, р-н Хорошево-Мневники, м. Бульвар Генерала Карбышева, Живописная улица, 6К3',
 'Москва, СЗАО, р-н Хорошево-Мневники, м. Бульвар Генерала Карбышева, Живописная улица, 6К3',
 'Москва, СЗАО, р-н Хорошево-Мневники, м. Бульвар Генерала Карбышева, Живописная улица, 6К3']

In [113]:
list(matches_base['Адрес_реновация'][0:20])

['проспект Мира, дом 188А',
 'проспект Мира, дом 188А',
 'улица Дмитрия Ульянова, дом 47, корпус 2',
 'бульвар Генерала Карбышева, дом 6, корпус 1',
 'бульвар Генерала Карбышева, дом 6, корпус 2',
 'бульвар Генерала Карбышева, дом 6, корпус 3',
 'бульвар Генерала Карбышева, дом 6, корпус 4']

## 5) Экспорт результатов
Сохраняем результат в эксельку для удобного просмотра и взаимодействия.

In [117]:
with pd.ExcelWriter("cian_renovation_matches.xlsx", engine="openpyxl") as xls:
    matches_base.to_excel(xls, sheet_name="base_match", index=False)
print("Готово")

Готово
